Setup Library to help with compiling and running

In [1]:
import os
import subprocess
import re
import numpy as np

def remove_non_hex(s):
    return re.sub(r'[^0-9A-Fa-f]', '', s)

def file_exists(file_path):
    return os.path.isfile(file_path)

def make_directory(destination_directory):
    # check that the destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

def copy_files_to_directory(files:list, destination):
    for file in files:
        if not file_exists(file): raise Exception('ERROR: cannot find file ' + str(file))
        subprocess.run(["cp", file, destination], capture_output=True, text=True)

Set settings, compile the C code, and program the Chip Whisperer  
  
Make sure to change the Firmware Directory to be correct

In [2]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_1_1'
FIRMWARE_DIR='../../../hardware/victims/firmware'
C_FOLDER_NAME='assignment1'
C_DIR = os.path.join(FIRMWARE_DIR, C_FOLDER_NAME)
%run "../../Setup_Scripts/Setup_Generic.ipynb"


INFO: Found ChipWhisperer😍


Setup the C Code Folder  
Code will be copied to the desired folder

In [3]:
#setup the folder for the c code
make_directory(C_DIR)
files_to_copy = ['./assignment1.c','./makefile']
copy_files_to_directory(files_to_copy, C_DIR)
print("C_DIR: ", C_DIR)


C_DIR:  ../../../hardware/victims/firmware\assignment1


The code below compiles the code and programs the chip.  
Make sure that the directory path matches C_DIR

In [4]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
#make sure path below matches C_DIR
cd ../../../hardware/victims/firmware/assignment1
echo $PWD
#remove all old generated files
rm -rf ./.dep
rm -rf ./objdir-CWNANO
rm ./*CWNANO*
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

/home/portable/chipwhisperer/hardware/victims/firmware/assignment1
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
mkdir .dep
Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10-2020-q4-major) 10.2.1 20201103 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir objdir-CWNANO 
.
Welcome to another exciting ChipWhisperer target build!!
.
Assembling: .././hal/stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x a

assignment1.c: In function 'decrypt':
assignment1.c:496:16: warning: passing argument 1 of 'expand_key' from incompatible pointer type [-Wincompatible-pointer-types]
  496 |     expand_key(key, keys);
      |                ^~~
      |                |
      |                uint8_t (*)[4] {aka unsigned char (*)[4]}
assignment1.c:356:26: note: expected 'uint8_t *' {aka 'unsigned char *'} but argument is of type 'uint8_t (*)[4]' {aka 'unsigned char (*)[4]'}
  356 | void expand_key(uint8_t *master_key, uint8_t keys[11][4][4]){
      |                 ~~~~~~~~~^~~~~~~~~~
assignment1.c: In function 'send_message':
assignment1.c:534:27: warning: incompatible implicit declaration of built-in function 'strlen'
  534 |     simpleserial_put('r', strlen(msg), (uint8_t *)msg);
      |                           ^~~~~~
assignment1.c:25:1: note: include '<string.h>' or provide a declaration of 'strlen'
   24 | #include <stdio.h>
  +++ |+#include <string.h>
   25 | 
assignment1.c: In function 'decryp

.
Compiling:
    .././hal/stm32f0_nano/stm32f0_hal_nano.c ...Done!
.
Compiling:
    .././hal/stm32f0/stm32f0_hal_lowlevel.c ...Done!
.
LINKING:
    assignment1-CWNANO.elf ...Done!
.
Creating load file for Flash: assignment1-CWNANO.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature assignment1-CWNANO.elf assignment1-CWNANO.hex
.
Creating load file for Flash: assignment1-CWNANO.bin
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature assignment1-CWNANO.elf assignment1-CWNANO.bin
.
Creating load file for EEPROM: assignment1-CWNANO.eep
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex assignment1-CWNANO.elf assignment1-CWNANO.eep || exit 0
.
Creating Symbol Table: assignment1-CWNANO.sym
arm-none-eabi-nm -n assignment1-CWNANO.elf > assignment1-CWNANO.sym
.
Creating Extended Listing: assignment1-CWNANO.lss
arm-none-eabi-objdump -h -S -z assignment1-CWNANO.elf 

Program the ChipWhisperer

In [5]:
hexfile = 'assignment1-{}.hex'.format(PLATFORM)
hexfilepath = str(os.path.join(C_DIR, hexfile))
print(hexfilepath)
cw.program_target(scope, prog, hexfilepath.format(PLATFORM))


../../../hardware/victims/firmware\assignment1\assignment1-CWNANO.hex
Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 11035 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 11035 bytes


Library for using the AES

In [6]:
import numpy as np

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array. """
    return bytes(matrix.flatten())  # Flattening ensures it's a 1D array
def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def printmatrix(a):
    print('\n', end='')
    for column in a:
        for item in column:
            print(hex(item)[2:], end=' ')
        print('\n', end='')

def set_key_cw(hexkey: bytearray, print_out=False):
    target.simpleserial_write('k', hexkey)
    keyresponse = target.simpleserial_read('r', 1)
    if print_out:
        if keyresponse[0] == 0x00:
            print("successfully set key to")
            print(hexkey.hex())
        else:
            print("failed to set key")
    return keyresponse
    
def encrypt_plaintext_cw(hexvalue: bytearray, print_out=False):
    target.simpleserial_write('p', hexvalue)
    testresponse = target.simpleserial_read('r', 16)
    if print_out:
        print('plain text input: ')
        print(hexvalue.hex())
        print('aes encryption output:')
        print(testresponse.hex())
    return testresponse

def decrypt_plaintext_cw(hexvalue: bytearray, print_out=False):
    target.simpleserial_write('c', hexvalue)
    testresponse = target.simpleserial_read('r', 16)
    if print_out:
        print('cipher input: ')
        print(hexvalue.hex())
        print('aes decryption output:')
        print(testresponse.hex())
    return testresponse

# Tests #


In [7]:
set_key_cw(bytearray.fromhex("117e111128aed2a6abf7158809cf4fac"), print_out=True) 
print('\n', end='')
encrypt_plaintext_cw(bytearray.fromhex("00 01 02 03 04 05 06 07 08 09 0A 0B 0C 0D 0E 0F"), print_out=True)
print('\n', end='')
decrypt_plaintext_cw(bytearray.fromhex("8a5bfe6ef26542a1defbe6be47313e02"), print_out=True)

successfully set key to
117e111128aed2a6abf7158809cf4fac

plain text input: 
000102030405060708090a0b0c0d0e0f
aes encryption output:
8a5bfe6ef26542a1defbe6be47313e02

cipher input: 
8a5bfe6ef26542a1defbe6be47313e02
aes decryption output:
000102030405060708090a0b0c0d0e0f


bytearray(b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f')

In [8]:
set_key_cw(bytearray.fromhex("00 11 22 33 44 55 66 77 88 99 AA BB CC DD EE FF"), print_out=True) 
print('\n', end='')
encrypt_plaintext_cw(bytearray.fromhex("11 31 43 88 49 28 AC DC BB BD F9 7F B9 03 91 77"), print_out=True)
print('\n', end='')
decrypt_plaintext_cw(bytearray.fromhex("cba9d25c2ae56657241e177a430f6151"), print_out=True)

successfully set key to
00112233445566778899aabbccddeeff

plain text input: 
113143884928acdcbbbdf97fb9039177
aes encryption output:
cba9d25c2ae56657241e177a430f6151

cipher input: 
cba9d25c2ae56657241e177a430f6151
aes decryption output:
113143884928acdcbbbdf97fb9039177


bytearray(b'\x111C\x88I(\xac\xdc\xbb\xbd\xf9\x7f\xb9\x03\x91w')

In [10]:
set_key_cw(bytearray.fromhex("FF FF FF FF FF FF FF FF FF 00 00 00 00 00 00 00"), print_out=True) 
print('\n', end='')
encrypt_plaintext_cw(bytearray.fromhex("AB CD 11 30 4D 3D 2D 1D EF EE ED EC AB AC AD AE"), print_out=True)
print('\n', end='')
decrypt_plaintext_cw(bytearray.fromhex("72cc50002127e0764da710782898daff"), print_out=True)

successfully set key to
ffffffffffffffffff00000000000000

plain text input: 
abcd11304d3d2d1defeeedecabacadae
aes encryption output:
72cc50002127e0764da710782898daff

cipher input: 
72cc50002127e0764da710782898daff
aes decryption output:
abcd11304d3d2d1defeeedecabacadae


bytearray(b'\xab\xcd\x110M=-\x1d\xef\xee\xed\xec\xab\xac\xad\xae')